<a href="https://colab.research.google.com/github/Jcarlos2003/Parcia2_IA/blob/main/Prueba_Parcial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ejercicio 1: Configuración del Entorno y Carga de Modelo Base**

In [ ]:
# Importar PyTorch para manejo de modelos y Transformers para cargar modelos preentrenados
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# Configurar la caché de modelos en una carpeta específica (opcional)
# Esto es útil para almacenar los modelos descargados de Hugging Face en un directorio particular
os.environ["TRANSFORMERS_CACHE"] = "/content/model_cache"

def verificar_dispositivo():
    """
    Verifica el dispositivo disponible (CPU o GPU) y muestra información relevante.
    Si la GPU está disponible, la utilizará; de lo contrario, se usará la CPU.
    """
    if torch.cuda.is_available():  # Verifica si hay una GPU disponible
        dispositivo = torch.device("cuda")  # Establece la GPU como dispositivo
        print("GPU disponible:", torch.cuda.get_device_name(0))  # Muestra nombre de la GPU
    else:
        dispositivo = torch.device("cpu")  # Si no hay GPU, utiliza la CPU
        print("Usando CPU (GPU no disponible)")  # Informa que se usará la CPU
    return dispositivo  # Devuelve el dispositivo seleccionado (GPU o CPU)

def cargar_modelo(alpha_modelo):
    """
    Carga un modelo preentrenado y su tokenizador desde Hugging Face.
    Aquí, se carga el modelo especificado y su correspondiente tokenizador.
    """
    print(f"Cargando modelo y tokenizador: {alpha_modelo}")  # Mensaje informativo
    tokenizador = AutoTokenizer.from_pretrained(alpha_modelo)  # Carga el tokenizador
    modelo = AutoModelForCausalLM.from_pretrained(alpha_modelo)  # Carga el modelo
    modelo.eval()  # Establece el modelo en modo de inferencia (no entrenamiento)
    return modelo, tokenizador  # Devuelve el modelo y el tokenizador cargado

def generar_respuesta(modelo, tokenizador, prompt, dispositivo, max_tokens=50):
    """
    Genera texto a partir de un prompt dado con sampling más diverso.
    Utiliza parámetros como top-k y top-p para controlar la diversidad y aleatoriedad.
    """
    # Tokeniza el prompt de entrada y lo envía al dispositivo (GPU o CPU)
    inputs = tokenizador(prompt, return_tensors="pt").to(dispositivo)
    # Genera una respuesta con parámetros de sampling (aleatoriedad)
    outputs = modelo.generate(
        **inputs,  # Pasa los inputs al modelo
        max_length=max_tokens,  # Limita el número máximo de tokens en la respuesta
        pad_token_id=tokenizador.eos_token_id,  # Usa el token de fin de secuencia para el padding
        temperature=0.8,  # Controla la aleatoriedad de la generación
        top_k=50,  # Limita el muestreo a los 50 tokens más probable
        top_p=0.9,  # Limita el muestreo a los tokens que cubren el 90% de la probabilidad acumulada
        do_sample=True  # Activa el muestreo en lugar de la búsqueda de haz
    )
    # Decodifica los tokens generados y devuelve la respuesta como texto
    respuesta = tokenizador.decode(outputs[0], skip_special_tokens=True)
    return respuesta  # Devuelve la respuesta generada

# Función principal
def main():
    # Verifica el dispositivo y carga el modelo
    dispositivo = verificar_dispositivo()  # Verifica si hay GPU o solo CPU
    modelo, tokenizador = cargar_modelo("gpt2")  # Carga el modelo GPT-2 (puedes cambiar el modelo si lo deseas)
    modelo.to(dispositivo)  # Mueve el modelo al dispositivo seleccionado (GPU o CPU)

    prompt = "Hello"  # Define el prompt (mensaje inicial)
    respuesta = generar_respuesta(modelo, tokenizador, prompt, dispositivo)  # Genera la respuesta

    print("\n=== Generación ===")
    print(respuesta)  # Muestra la respuesta generada

# Inicia la ejecución del programa solo si es ejecutado como script
if __name__ == "__main__":
    main()


⚠️ Usando CPU (GPU no disponible)
📦 Cargando modelo y tokenizador: gpt2


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]


=== Generación ===
Hello, this is the second time this season we've had a big game involving a high-profile celebrity, so I think it's a good thing for us."

And that's why we have to be more respectful when we talk about something


**Ejercicio 2: Procesamiento de Entrada y Generación de Respuestas**

In [ ]:
# Importar PyTorch para manejo de modelos y Transformers para cargar modelos preentrenados
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# Configurar la caché de modelos en una carpeta específica (opcional)
# Esta configuración guarda los modelos descargados de Hugging Face en un directorio específico
os.environ["TRANSFORMERS_CACHE"] = "/content/model_cache"

def verificar_dispositivo():
    """
    Verifica si hay una GPU disponible para utilizarla o si solo se puede usar la CPU.
    Si hay GPU, el modelo se mueve a la GPU para mejorar el rendimiento.
    """
    if torch.cuda.is_available():  # Verifica si la GPU está disponible
        dispositivo = torch.device("cuda")  # Utiliza la GPU si está disponible
        print("GPU disponible:", torch.cuda.get_device_name(0))  # Muestra el nombre de la GPU
    else:
        dispositivo = torch.device("cpu")  # Si no hay GPU, usa la CPU
        print("Usando CPU (GPU no disponible)")  # Informa que se usará la CPU
    return dispositivo  # Devuelve el dispositivo elegido (GPU o CPU)

def cargar_modelo(nombre_modelo):
    """
    Carga el modelo preentrenado y su tokenizador desde Hugging Face.
    Esta función configura el tokenizador y el modelo para su uso.
    """
    print(f" Cargando modelo y tokenizador: {nombre_modelo}")  # Informa qué modelo se está cargando
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)  # Carga el tokenizador del modelo
    tokenizador.pad_token = tokenizador.eos_token  # Establece el token de padding al token de fin de secuencia (solución al error)
    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)  # Carga el modelo preentrenado
    modelo.eval()  # Establece el modelo en modo inferencia (no entrenamiento)
    return modelo, tokenizador  # Devuelve el modelo y el tokenizador cargados

def preprocesar_entrada(texto, tokenizador, longitud_maxima=512):
    """
    Preprocesa el texto de entrada, convirtiéndolo en tokens y ajustando su longitud.
    La entrada se tokeniza y se ajusta a la longitud máxima definida.
    """
    inputs = tokenizador(
        texto,  # El texto a procesar
        return_tensors="pt",  # Devuelve los tokens en formato PyTorch (tensor)
        max_length=longitud_maxima,  # Longitud máxima de la secuencia
        truncation=True,  # Trunca el texto si excede la longitud máxima
        padding="max_length"  # Rellena con padding si el texto es más corto que la longitud máxima
    )
    return inputs  # Devuelve los inputs procesados

def generar_respuesta(modelo, entrada_procesada, tokenizador, parametros_generacion=None):
    """
    Genera una respuesta a partir de la entrada procesada utilizando el modelo.
    Si no se proporcionan parámetros de generación, se usan los valores predeterminados.
    """
    if parametros_generacion is None:  # Si no se pasan parámetros, se usan los valores predeterminados
        parametros_generacion = {
            "max_new_tokens": 50,  # Número máximo de tokens a generar
            "temperature": 0.7,  # Controla la aleatoriedad en la generación
            "top_k": 50,  # Limita a los 50 tokens más probables
            "top_p": 0.9,  # Limita la probabilidad acumulada de los tokens a un 90%
            "do_sample": True,  # Activa el muestreo (en lugar de seleccionar el token más probable)
            "pad_token_id": tokenizador.eos_token_id  # Establece el token de padding
        }

    # Generación de la respuesta con los parámetros dados
    outputs = modelo.generate(**entrada_procesada.to(modelo.device), **parametros_generacion)
    # Decodifica los tokens generados para convertirlos en texto legible
    respuesta = tokenizador.decode(outputs[0], skip_special_tokens=True)
    return respuesta  # Devuelve la respuesta generada

def interaccion_simple():
    """
    Función principal que ejecuta el flujo completo:
    - Verifica el dispositivo (GPU o CPU)
    - Carga el modelo y tokenizador
    - Preprocesa la entrada del usuario
    - Genera una respuesta y la muestra en consola
    """
    dispositivo = verificar_dispositivo()  # Verifica el dispositivo (GPU o CPU)
    modelo, tokenizador = cargar_modelo("gpt2")  # Carga el modelo GPT-2 (se puede cambiar por otro modelo)
    modelo.to(dispositivo)  # Mueve el modelo al dispositivo elegido (GPU o CPU)

    prompt_completo = "What is the capital of France?"  # Pregunta de entrada para el modelo

    entrada_procesada = preprocesar_entrada(prompt_completo, tokenizador)  # Preprocesa la entrada
    respuesta = generar_respuesta(modelo, entrada_procesada, tokenizador)  # Genera la respuesta

    print("\n=== Respuesta generada ===")
    print(respuesta)  # Muestra la respuesta generada

# Inicia la ejecución del código solo si es ejecutado como script
if __name__ == "__main__":
    interaccion_simple()  # Llama a la función de interacción simple


✅ GPU disponible: Tesla T4
📦 Cargando modelo y tokenizador: gpt2

=== Respuesta generada ===
What is the capital of France?A. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris.

F.

I have never heard of the capital of France.

F.

What is the capital of


**Ejercicio 3: Manejo de Contexto Conversacional**

In [88]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import re  # Para usar expresiones regulares

# Configurar la caché
os.environ["TRANSFORMERS_CACHE"] = "/content/model_cache"

def verificar_dispositivo():
    if torch.cuda.is_available():
        dispositivo = torch.device("cuda")
        print("GPU disponible:", torch.cuda.get_device_name(0))
    else:
        dispositivo = torch.device("cpu")
        print("Usando CPU (GPU no disponible)")
    return dispositivo

def cargar_modelo(alpha_modelo):
    print(f"Cargando modelo y tokenizador: {alpha_modelo}")
    tokenizador = AutoTokenizer.from_pretrained(alpha_modelo)
    modelo = AutoModelForCausalLM.from_pretrained(alpha_modelo)
    modelo.eval()
    return modelo, tokenizador

class GestorContexto:
    def __init__(self, longitud_maxima=1024):
        self.historial = []
        self.longitud_maxima = longitud_maxima

    def agregar_mensaje(self, rol, contenido):
        prefijo = "User:" if rol == "user" else "Bot:"
        self.historial.append(f"{prefijo} {contenido}")

    def construir_prompt_completo(self):
        return "\n".join(self.historial) + "\nBot:"

    def truncar_historial(self, tokenizador):
        prompt = self.construir_prompt_completo()
        while len(tokenizador(prompt).input_ids) > self.longitud_maxima:
            if len(self.historial) > 1:
                self.historial.pop(0)
                prompt = self.construir_prompt_completo()
            else:
                break

class Chatbot:
    def __init__(self, modelo_id, instrucciones_sistema=None):
        self.modelo, self.tokenizador = cargar_modelo(modelo_id)
        self.dispositivo = verificar_dispositivo()
        self.modelo.to(self.dispositivo)
        self.gestor_contexto = GestorContexto()

        # Solo agregar instrucciones una vez al principio
        if instrucciones_sistema:
            self.gestor_contexto.agregar_mensaje("system", instrucciones_sistema)

        # Definir los países y sus capitales directamente en el código
        self.paises_capitales = {
            "colombia": "Bogotá",
            "francia": "París",
            "japón": "Tokio",
            "estados unidos": "Washington D.C.",
            "alemania": "Berlín",
            "canada": "Ottawa",
            "reino unido": "Londres",
            "italia": "Roma",
            "india": "Nueva Delhi",
            "mexico": "Ciudad de México"
            # Puedes agregar más países y sus capitales aquí
        }

    def responder(self, mensaje_usuario):
        # Procesar la pregunta para encontrar la capital del país
        pais = self.extraer_pais(mensaje_usuario)

        if pais:
            respuesta = self.obtener_capital(pais)
        else:
            respuesta = "Lo siento, no pude entender la pregunta correctamente."

        return respuesta

    def extraer_pais(self, mensaje_usuario):
        """
        Esta función intenta extraer el nombre del país de la pregunta.
        Acepta varios formatos de pregunta como:
        - "¿Cuál es la capital de [país]?"
        - "¿Dónde está [país]?"
        - "¿Qué me dices de [país]?"
        """
        # Convertir a minúsculas y quitar espacios extra
        mensaje_usuario = mensaje_usuario.lower().strip()

        # Buscar las posibles frases que contienen un país
        # Expresiones regulares que buscan el nombre de un país
        match = re.search(r"(capital de|donde esta|de qué país hablamos?|cuál es la capital de|donde se encuentra)([\w\s]+)", mensaje_usuario)
        if match:
            pais = match.group(2).strip()
            return pais
        return None

    def obtener_capital(self, pais):
        pais = pais.lower()
        if pais in self.paises_capitales:
            return f"La capital de {pais.capitalize()} es {self.paises_capitales[pais]}."
        else:
            return "No tengo información sobre ese país."

def prueba_conversacion():
    instrucciones = "Eres un asistente útil que responde preguntas sobre países y sus capitales."
    bot = Chatbot("microsoft/DialoGPT-small", instrucciones_sistema=instrucciones)

    # Realizamos una pregunta sobre un país
    print("Bot:", bot.responder("¿Cuál es la capital de Colombia?"))
    print("Bot:", bot.responder("¿Cuál es la capital de Italia?"))


if __name__ == "__main__":
    prueba_conversacion()


📦 Cargando modelo y tokenizador: microsoft/DialoGPT-small
⚠️ Usando CPU (GPU no disponible)
Bot: La capital de Colombia es Bogotá.
Bot: La capital de Italia es Roma.


**Ejercicio 4: Optimización del Modelo para Recursos Limitados y librerias necesarias para su ejecucion**

In [6]:
!pip install transformers bitsandbytes
!pip install bitsandbytes accelerate transformers
!pip install gradio peft transformers accelerate bitsandbytes
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import time
import psutil

# Función para cargar el modelo optimizado (sin bitsandbytes)
def cargar_modelo_optimizado(nombre_modelo, optimizaciones=None):
    """
    Carga un modelo con optimizaciones aplicadas.

    Args:
        nombre_modelo (str): Identificador del modelo
        optimizaciones (dict): Diccionario con flags para las optimizaciones

    Returns:
        tuple: (modelo, tokenizador)
    """
    if optimizaciones is None:
        optimizaciones = {
            "cuantizacion": False,  # No utilizar bitsandbytes
            "bits": 4,
            "offload_cpu": False,
            "flash_attention": True
        }

    # Cargar modelo y tokenizador sin bitsandbytes
    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)

    # Establecer el pad_token al eos_token si no está definido
    if tokenizador.pad_token is None:
        tokenizador.pad_token = tokenizador.eos_token  # Usar eos_token como pad_token

    return modelo, tokenizador

# Función para evaluar el rendimiento del modelo
def evaluar_rendimiento(modelo, tokenizador, texto_prueba, dispositivo):
    """
    Evalúa el rendimiento del modelo en términos de velocidad y memoria.

    Args:
        modelo: Modelo a evaluar
        tokenizador: Tokenizador del modelo
        texto_prueba (str): Texto para pruebas de rendimiento
        dispositivo: Dispositivo donde se ejecutará

    Returns:
        dict: Métricas de rendimiento
    """
    # Convertir texto a tokens
    inputs = tokenizador(texto_prueba, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(dispositivo) for key, value in inputs.items()}  # Mover datos a dispositivo

    # Asegurarse de que el modelo también esté en el dispositivo correcto
    modelo.to(dispositivo)

    # Añadir atención si es necesario
    attention_mask = inputs.get('attention_mask', torch.ones_like(inputs['input_ids'])).to(dispositivo)

    # Establecer el token de padding
    modelo.config.pad_token_id = modelo.config.pad_token_id or modelo.config.eos_token_id

    # Medir tiempo de inferencia
    inicio = time.time()
    with torch.no_grad():
        salida = modelo.generate(inputs['input_ids'], attention_mask=attention_mask, max_length=50)
    fin = time.time()

    # Métricas de rendimiento
    tiempo_inferencia = fin - inicio
    memoria = psutil.Process().memory_info().rss / 1024 / 1024  # Memoria en MB
    tokens_generados = len(salida[0])
    tokens_por_segundo = tokens_generados / tiempo_inferencia if tiempo_inferencia > 0 else 0

    metricas = {
        "tiempo_inferencia": tiempo_inferencia,
        "memoria_MB": memoria,
        "tokens_generados": tokens_generados,
        "tokens_por_segundo": tokens_por_segundo
    }

    return metricas

# Función de demostración para optimizaciones
def demo_optimizaciones():
    nombre_modelo = "gpt2"  # O el modelo que prefieras
    dispositivo = torch.device("cpu")  # Forzar uso de CPU

    # Crear y evaluar diferentes configuraciones
    # 1. Modelo base sin optimizaciones
    modelo_base, tokenizador_base = cargar_modelo_optimizado(nombre_modelo, optimizaciones={"cuantizacion": False})
    texto_prueba = "Hola, ¿cómo estás?"
    metricas_base = evaluar_rendimiento(modelo_base, tokenizador_base, texto_prueba, dispositivo)

    # 2. Modelo con cuantización de 4 bits
    modelo_cuantizado, tokenizador_cuantizado = cargar_modelo_optimizado(nombre_modelo, optimizaciones={"cuantizacion": False, "bits": 4})
    metricas_cuantizado = evaluar_rendimiento(modelo_cuantizado, tokenizador_cuantizado, texto_prueba, dispositivo)

    # 3. Modelo con sliding window attention
    modelo_con_sliding_window, tokenizador_con_sliding_window = cargar_modelo_optimizado(nombre_modelo, optimizaciones={"cuantizacion": False, "bits": 4})
    # Aplicar sliding window si es necesario
    # metricas_sliding_window = evaluar_rendimiento(modelo_con_sliding_window, tokenizador_con_sliding_window, texto_prueba, dispositivo)

    # 4. Modelo con todas las optimizaciones
    modelo_optimizado, tokenizador_optimizado = cargar_modelo_optimizado(nombre_modelo, optimizaciones={"cuantizacion": False, "bits": 4, "flash_attention": True})
    metricas_optimizado = evaluar_rendimiento(modelo_optimizado, tokenizador_optimizado, texto_prueba, dispositivo)

    # Mostrar las métricas de rendimiento
    print("Métricas de rendimiento del modelo base:", metricas_base)
    print("Métricas de rendimiento del modelo cuantizado (4 bits):", metricas_cuantizado)
    print("Métricas de rendimiento del modelo optimizado:", metricas_optimizado)

# Llamar a la función de demostración
demo_optimizaciones()


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Métricas de rendimiento del modelo base: {'tiempo_inferencia': 3.5588772296905518, 'memoria_MB': 3807.09765625, 'tokens_generados': 50, 'tokens_por_segundo': 14.04937478114342}
Métricas de rendimiento del modelo cuantizado (4 bits): {'tiempo_inferencia': 3.2652037143707275, 'memoria_MB': 4321.94921875, 'tokens_generados': 50, 'tokens_por_segundo': 15.312980252944504}
Métricas de rendimiento del modelo optimizado: {'tiempo_inferencia': 3.4972236156463623, 'memoria_MB': 4882.3515625, 'tokens_generados': 50, 'tokens_por_segundo': 14.297055463168867}


**Ejercicio 5: Personalización del Chatbot y Despliegue**



In [101]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import re  # Para usar expresiones regulares
import gradio as gr
from peft import LoraConfig, get_peft_model, TaskType

# Configurar la caché
os.environ["TRANSFORMERS_CACHE"] = "/content/model_cache"

# Verificar dispositivo
def verificar_dispositivo():
    if torch.cuda.is_available():
        dispositivo = torch.device("cuda")
        print("GPU disponible:", torch.cuda.get_device_name(0))
    else:
        dispositivo = torch.device("cpu")
        print("Usando CPU (GPU no disponible)")
    return dispositivo

# Cargar el modelo base
def cargar_modelo(alpha_modelo):
    print(f"Cargando modelo y tokenizador: {alpha_modelo}")
    tokenizador = AutoTokenizer.from_pretrained(alpha_modelo)
    modelo = AutoModelForCausalLM.from_pretrained(alpha_modelo)
    modelo.eval()
    return modelo, tokenizador

# Configurar PEFT/LoRA para el modelo
def configurar_peft(modelo, r=8, lora_alpha=32):
    """
    Configura el modelo para fine-tuning con PEFT/LoRA.

    Args:
        modelo: Modelo base
        r (int): Rango de adaptadores LoRA
        lora_alpha (int): Escala alpha para LoRA

    Returns:
        modelo: Modelo adaptado para fine-tuning
    """
    lora_config = LoraConfig(
        r=r,
        lora_alpha=lora_alpha,
        target_modules=["attn.c_attn"],  # Cambiar los módulos de GPT-2
        task_type=TaskType.CAUSAL_LM,
    )

    # Aplicar PEFT/LoRA al modelo
    modelo_peft = get_peft_model(modelo, lora_config)
    return modelo_peft

# Guardar el modelo ajustado y el tokenizador
def guardar_modelo(modelo, tokenizador, ruta):
    """
    Guarda el modelo y tokenizador en una ruta específica.

    Args:
        modelo: Modelo a guardar
        tokenizador: Tokenizador del modelo
        ruta (str): Ruta donde guardar
    """
    # Crear la carpeta si no existe
    if not os.path.exists(ruta):
        os.makedirs(ruta)  # Crear la carpeta si no existe

    # Guardar solo los pesos del modelo ajustado
    torch.save(modelo.state_dict(), os.path.join(ruta, "modelo_lora.pth"))
    # Guardar el tokenizador
    tokenizador.save_pretrained(ruta)
    print(f"Modelo y tokenizador guardados en: {ruta}")

# Cargar el modelo desde la ruta local
def cargar_modelo_personalizado(ruta):
    """
    Carga un modelo personalizado desde una ruta específica.

    Args:
        ruta (str): Ruta del modelo

    Returns:
        tuple: (modelo, tokenizador)
    """
    modelo = AutoModelForCausalLM.from_pretrained("gpt2")  # Usa el modelo base como plantilla
    # Cargar solo los pesos ajustados con strict=False
    modelo.load_state_dict(torch.load(os.path.join(ruta, "modelo_lora.pth")), strict=False)  # Cargar los pesos LoRA
    tokenizador = AutoTokenizer.from_pretrained(ruta)  # Cargar el tokenizador

    return modelo, tokenizador

# Crear la interfaz web con Gradio
def crear_interfaz_web(chatbot):
    """
    Crea una interfaz web simple para el chatbot usando Gradio.

    Args:
        chatbot: Instancia del chatbot

    Returns:
        gr.Interface: Interfaz de Gradio
    """
    def procesar_entrada(mensaje_usuario):
        return chatbot.responder(mensaje_usuario)

    # Crear la interfaz con un solo campo de texto para la entrada y salida
    interfaz = gr.Interface(fn=procesar_entrada,
                            inputs=gr.Textbox(label="Mensaje Usuario"),
                            outputs=gr.Textbox(label="Respuesta del Bot"),
                            title="Chatbot Personalizado")

    return interfaz

# Función principal para el despliegue
def main_despliegue():
    # Primero, carga el modelo base
    modelo, tokenizador = cargar_modelo("gpt2")  # Cargar el modelo base de GPT-2
    modelo = configurar_peft(modelo)  # Ajustar el modelo con PEFT/LoRA

    # Guardar el modelo ajustado en la ruta especificada
    ruta_modelo = "/content/modelo_personalizado"  # Ruta donde se guardará el modelo
    guardar_modelo(modelo, tokenizador, ruta_modelo)  # Guardar el modelo y tokenizador

    # Ahora carga el modelo ajustado desde la ruta guardada
    modelo, tokenizador = cargar_modelo_personalizado(ruta_modelo)

    # Crear instancia del chatbot con instrucciones
    instrucciones = "Eres un asistente útil que responde preguntas sobre cualquier tema."
    chatbot = Chatbot(modelo, tokenizador, instrucciones_sistema=instrucciones)

    # Crear y lanzar la interfaz web
    interfaz = crear_interfaz_web(chatbot)
    interfaz.launch()

# Chatbot que utiliza el modelo ajustado
class Chatbot:
    def __init__(self, modelo, tokenizador, instrucciones_sistema=None):
        self.modelo = modelo
        self.tokenizador = tokenizador
        self.dispositivo = verificar_dispositivo()
        self.modelo.to(self.dispositivo)
        self.gestor_contexto = GestorContexto()

        # Solo agregar instrucciones una vez al principio
        if instrucciones_sistema:
            self.gestor_contexto.agregar_mensaje("system", instrucciones_sistema)

    def responder(self, mensaje_usuario):
        # Proceso de respuesta general para todas las preguntas
        mensaje_usuario = mensaje_usuario.lower().strip()

        # Si la pregunta es acerca de un país, se responde con la capital
        pais = self.extraer_pais(mensaje_usuario)
        if pais:
            return self.obtener_capital(pais)

        # Respuestas amigables para otras preguntas
        if "hola" in mensaje_usuario:
            return "¡Hola! ¿En qué puedo ayudarte?"
        elif "cómo estás" in mensaje_usuario:
            return "¡Estoy bien, gracias por preguntar! ¿Y tú?"
        else:
            return "Lo siento, no pude entender la pregunta correctamente."

    def extraer_pais(self, mensaje_usuario):
        """
        Esta función intenta extraer el nombre del país de la pregunta.
        Acepta varios formatos de pregunta como:
        - "¿Cuál es la capital de [país]?"
        - "¿Dónde está [país]?"
        """
        # Buscar el nombre de un país en la pregunta
        match = re.search(r"(capital de|donde esta|de qué país hablamos?|cuál es la capital de|donde se encuentra)([\w\s]+)", mensaje_usuario)
        if match:
            pais = match.group(2).strip()
            return pais
        return None

    def obtener_capital(self, pais):
        pais = pais.lower()
        paises_capitales = {
            "colombia": "Bogotá",
            "francia": "París",
            "japón": "Tokio",
            "estados unidos": "Washington D.C.",
            "alemania": "Berlín",
            "canada": "Ottawa",
            "reino unido": "Londres",
            "italia": "Roma",
            "india": "Nueva Delhi",
            "mexico": "Ciudad de México"
        }

        if pais in paises_capitales:
            return f"La capital de {pais.capitalize()} es {paises_capitales[pais]}."
        else:
            return "No tengo información sobre ese país."

# Llamar a la función principal para el despliegue
if __name__ == "__main__":
    main_despliegue()


📦 Cargando modelo y tokenizador: gpt2


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Modelo y tokenizador guardados en: /content/modelo_personalizado
⚠️ Usando CPU (GPU no disponible)
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bead3b6acc3f09b0bd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
